In [3]:
!pip3 install -q -U tiktoken python-dotenv tabulate

In [181]:
import sys
import os
import json
import pandas as pd
import numpy as np
import ast
# import altair as alt
import matplotlib.pyplot as plt
# import altair_catplot as altcat
# import seaborn as sns
# import wikipedia
# from wikidata.client import Client
# import wikipediaapi
import uuid
import re
import csv
# import jsonlines
import math
import random
import requests
import gzip
from typing import List, Any, Dict, Tuple
import tempfile
import copy
import string
import subprocess
import typing
import time
from datetime import datetime
# import tweepy
from collections import Counter, defaultdict
from scipy import stats
from tabulate import tabulate


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:
# from src.helpers import gpt
# from src.scripts import load_datasets
# from src.helpers import io
from src.scripts import run_fake_data_test
from src.helpers.visualisation import barplot_distribution, plot_confusion_matrix, tabulate_annotation_pair_summary, analyze_pair_annotations


## Run Analysis

In [183]:
dset = run_fake_data_test.run_test_cedric()

Loading conversations from data/sample120.json
Loaded 120 conversations.
Added conversation IDs to 9 files
Split annotations into two folders:
  - data/labelstudio_outputs_split1/: Contains 115 annotations with unique conversation IDs
  - data/labelstudio_outputs_split2/: Contains 63 annotations with duplicate conversation IDs
Adding AnnotationSet 'split1' for label=`media_format` (message-level): 220 of 1195 dataset messages, or 113 of 120 dataset conversations.
Adding AnnotationSet 'split1' for label=`topic` (message-level): 220 of 1195 dataset messages, or 112 of 120 dataset conversations.
Adding AnnotationSet 'split1' for label=`function_purpose` (message-level): 218 of 1195 dataset messages, or 112 of 120 dataset conversations.
Adding AnnotationSet 'split1' for label=`multi_turn_relationship` (message-level): 218 of 1195 dataset messages, or 112 of 120 dataset conversations.
Adding AnnotationSet 'split1' for label=`media_format_response` (message-level): 216 of 1195 dataset messag

In [184]:
# dset.data[0].to_dict()

In [185]:
def run_interrater_comparison(dataset, task_name):
    info_to_plot1 = dset.get_annotation_distribution(name=task_name, level="message", annotation_source='split1')
    info_to_plot2 = dset.get_annotation_distribution(name=task_name, level="message", annotation_source='split2')
    info_to_plot1b = dset.get_annotation_distribution(name=task_name, level="message", annotation_source='split1', annotation_as_list_type=True)
    info_to_plot2b = dset.get_annotation_distribution(name=task_name, level="message", annotation_source='split2', annotation_as_list_type=True)

    outdir = f"data/annotation_analysis_v0/{task_name}"
    os.makedirs(outdir, exist_ok=True)
    fig = barplot_distribution(
        {"Split1": info_to_plot1, "Split2": info_to_plot2}, normalize=True, 
        xlabel=task_name, ylabel="Proportion", title="",
        output_path=f"{outdir}/barchart.png", order="descending")
    
    fig_b = barplot_distribution(
        {"Split1": info_to_plot1b, "Split2": info_to_plot2b}, normalize=True, 
        xlabel=task_name, ylabel="Proportion", title="",
        output_path=f"{outdir}/multilabel_barchart.png", order="descending")

    info_to_plot_cm, agreement_metrics, paired_values = dataset.get_joint_distribution(
        annotations1=(task_name, 'split1'), 
        annotations2=(task_name, 'split2'), 
        level="message",
        compute_disagreement=True,
        verbose=True
    )

    fig2 = plot_confusion_matrix(
        info_to_plot_cm, normalize=True, xlabel="", ylabel="", title="Confusion Matrix",
        output_path=f"{outdir}/confusion_matrix.png")

    # print(paired_values[0:3])
    df = analyze_pair_annotations(paired_values)
    df.to_csv(f"{outdir}/pair_frequencies.csv", index=False, quoting=csv.QUOTE_NONNUMERIC)

    print()
    print(f"-----------------{task_name}-----------------")
    print(agreement_metrics)
    print(tabulate_annotation_pair_summary(df, 20))
    print(len(df))
    print()
    return paired_values

prompt_fields = [
    "multi_turn_relationship",
    "media_format",
    "topic",
    "function_purpose",
    "anthropomorphization",
    "restricted_flags",
]
response_fields = [
    "answer_form",
    "self_disclosure",
    "topic_response",
    "media_format_response",
    "restricted_flags_response",
]

task_annotations = {}
for feature in prompt_fields:
    task_annotations[feature] = run_interrater_comparison(dset, feature)
    # break
for feature in response_fields:
    task_annotations[feature] = run_interrater_comparison(dset, feature)
    # break

Found 218 items with `split1-multi_turn_relationship`, and 107 with `split2-multi_turn_relationship`.
Generated 104 label-level pairs (at level=message) out of 1195 total items.

-----------------multi_turn_relationship-----------------
{'label_agreement_rate': 0.8365384615384616, 'cohens_kappa': 0.7250388802488337}
+----------------------------+----------------------------+---------+------------------+------------+
| val1                       | val2                       |   count |   percent_of_all | matching   |
+============================+============================+=========+==================+============+
| ['First request']          | ['First request']          |      61 |            58.65 | T          |
+----------------------------+----------------------------+---------+------------------+------------+
| ['Related request']        | ['Related request']        |      10 |             9.62 | T          |
+----------------------------+----------------------------+---------+-

/Users/shayne/Documents/research/naturalistic-data/naturalistic-ai/src/helpers/visualisation.py:123: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  


Found 220 items with `split1-media_format`, and 106 with `split2-media_format`.
Generated 105 label-level pairs (at level=message) out of 1195 total items.

-----------------media_format-----------------
{'label_agreement_rate': 0.8380952380952381, 'cohens_kappa': 0.4690660321237359}
+--------------------------------------------+---------------------------------+---------+------------------+------------+
| val1                                       | val2                            |   count |   percent_of_all | matching   |
+============================================+=================================+=========+==================+============+
| ['Natural language']                       | ['Natural language']            |      81 |            77.14 | T          |
+--------------------------------------------+---------------------------------+---------+------------------+------------+
| ['For enu', 'Nat lan']                     | ['Natural language']            |       9 |          

/Users/shayne/Documents/research/naturalistic-data/naturalistic-ai/src/helpers/visualisation.py:123: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  


Found 220 items with `split1-topic`, and 108 with `split2-topic`.
Generated 106 label-level pairs (at level=message) out of 1195 total items.

-----------------topic-----------------
{'label_agreement_rate': 0.3113207547169811, 'cohens_kappa': 0.2645884812773237}
+---------------------------------------+---------------------------------------------+---------+------------------+------------+
| val1                                  | val2                                        |   count |   percent_of_all | matching   |
+=======================================+=============================================+=========+==================+============+
| ['Tec Sof & Com']                     | ['Tec Sof & Com']                           |      10 |             9.43 | T          |
+---------------------------------------+---------------------------------------------+---------+------------------+------------+
| ['Literature & Writing']              | ['Literature & Writing']                    

...

Found 201 items with `split1-topic_response`, and 107 with `split2-topic_response`.
Generated 95 label-level pairs (at level=message) out of 1195 total items.

-----------------topic_response-----------------
{'label_agreement_rate': 0.3263157894736842, 'cohens_kappa': 0.28284972870960134}
+---------------------------------------+----------------------------------------------+---------+------------------+------------+
| val1                                  | val2                                         |   count |   percent_of_all | matching   |
+=======================================+==============================================+=========+==================+============+
| ['Literature & Writing']              | ['Literature & Writing']                     |       7 |             7.37 | T          |
+---------------------------------------+----------------------------------------------+---------+------------------+------------+
| ['Eng & Inf', 'Tec Sof & Com']        | ['Tec Sof & 

/Users/shayne/Documents/research/naturalistic-data/naturalistic-ai/src/helpers/visualisation.py:123: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  


Found 216 items with `split1-media_format_response`, and 106 with `split2-media_format_response`.
Generated 102 label-level pairs (at level=message) out of 1195 total items.

-----------------media_format_response-----------------
{'label_agreement_rate': 0.6078431372549019, 'cohens_kappa': 0.41938238223993163}
+---------------------------------------------------+--------------------------------------+---------+------------------+------------+
| val1                                              | val2                                 |   count |   percent_of_all | matching   |
+===================================================+======================================+=========+==================+============+
| ['Natural language']                              | ['Natural language']                 |      38 |            37.25 | T          |
+---------------------------------------------------+--------------------------------------+---------+------------------+------------+
| ['For enu'

/Users/shayne/Documents/research/naturalistic-data/naturalistic-ai/src/helpers/visualisation.py:123: UserWarning: constrained_layout not applied because axes sizes collapsed to zero.  Try making figure larger or axes decorations smaller.
  


Found 29 items with `split1-restricted_flags_response`, and 15 with `split2-restricted_flags_response`.
Generated 2 label-level pairs (at level=message) out of 1195 total items.

-----------------restricted_flags_response-----------------
{'label_agreement_rate': 0.5, 'cohens_kappa': 0.0}
+-------------------------+-------------------------+---------+------------------+------------+
| val1                    | val2                    |   count |   percent_of_all | matching   |
+=========================+=========================+=========+==================+============+
| ['Sex exp con fic per'] | ['Sex exp con fic per'] |       1 |               50 | T          |
+-------------------------+-------------------------+---------+------------------+------------+
| ['Other']               | ['Sex exp con fic per'] |       1 |               50 | F          |
+-------------------------+-------------------------+---------+------------------+------------+
2



In [169]:

# non_matching_ids = [x[0] for x in task_annotations["media_format"] if x[1] != x[2]]
# dset.id_lookup(non_matching_ids, level="message")